# Budge interpreter in Python

We rely on Python lists data structure to represent sequences.

In [1]:
def sign(x):
  return 1 if x >= 1 else -1

def p(x):
  (n, c) = (1, 0)
  while c < x:
    n += 1
    for i in range(2, n+1):
      if n % i == 0:
        break
    if i == n:
      c += 1
  return n

def evaluate(i, code):
  for el in code:
    if isinstance(el, list) and len(el) > 1:
      while i % p(abs(el[0])) == 0:
        i = evaluate(i, el[1:])
    elif sign(el) == 1:
      i *= p(el)
    else:
      new_i = p(abs(el))
      if i % new_i == 0:
        i = int(i // new_i)

  return i

Use these helper methods to convert between a number and a list of registers (prime factorization):

In [2]:
def get_registers(i):
  (r, reg) = ({}, 1)
  while i != 1:
    while i % p(reg) == 0:
      if reg not in r: r[reg] = 0
      r[reg] += 1
      i //= p(reg)
    reg += 1
  return r

def set_registers(r):
  i = 1
  for reg in r:
    i *= p(reg) ** r[reg]
  return i

Example usage:

In [3]:
get_registers(12345)

{2: 1, 3: 1, 143: 1}

In [4]:
set_registers(get_registers(12345))

12345

# Examples

For example, to set register 1 to value 4, use the first prime $p(1) = 2$ and then power it to the value, i.e. $2^4$. To also set the second register to another value, say 5, use the second prime $p(2) = 3$ and power it to the value $3^5$. Multiply both registers like so to represent the value (4, 5) in memory: $2^4 \cdot 3^5$.

Then, to apply the code `[[2, -2, 1]]` to that input, use something like:

In [5]:
evaluate(2**4 * 3**5, [[2, -2, 1]])

512

The prime factorization of 512 is $2^9$, which indicates that the first register ($p(1) = 2$) has a value of 9, i.e. the addition of 4 and 5.

Or, easier with the helper methods:

In [6]:
get_registers(evaluate(set_registers({1: 4, 2: 5}), [[2, -2, 1]]))

{1: 9}

# Efficiency

A more efficient interpreter would be to use a `dict` for registers and directly adjust the values there, rather than relying on number division:

In [7]:
def fast_evaluate(i, code):
  for el in code:
    if isinstance(el, list) and len(el) > 1:
      while abs(el[0]) in i and i[abs(el[0])]:
        i = fast_evaluate(i, el[1:])
    else:
      (s, el) = (sign(el), abs(el))
      if el not in i: i[el] = 0
      i[el] = max(0, i[el] + s)
  # remove zero registers
  return {x:y for x,y in i.items() if y != 0}

The same example as before, using the `fast_evaluate` function:

In [8]:
fast_evaluate({1: 4, 2: 5}, [[2, -2, 1]])

{1: 9}

## Arithmetic

### Addition

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

The following code adds $x$ and $y$.

In [9]:
arith_add = [[2, -2, 1]]

In [10]:
fast_evaluate({1: 3, 2: 3}, arith_add)

{1: 6}

### Subtraction

*Input*: $2^x \cdot 3^y$. *Output*: $2^n \cdot 3^k$.

The following code will set $k$ to 1 if $y > x$, and 0 otherwise. The final result is $n = |x - y|$.

In [11]:
arith_sub = [
  [1, -1, 3, 5],   # move r1 to r3 r4
  [2, -2, 4, 6],   # move r2 to r5 r6
  [3, -3, -4],     # calculate r3 - r4
  [6, -5, -6],     # calculate r6 - r5
  [4, -4, 1, 3],   # move r4 to r1 and r3 (if it was set)
  [3, [3, -3], 2], # if r3 was set, then set r2 to 1 to indicate flag
  [5, -5, 1]]      # move r5 to r1 (if it was set)

In [12]:
fast_evaluate({1: 5, 2: 3}, arith_sub)

{1: 2}

In [13]:
fast_evaluate({1: 3, 2: 5}, arith_sub)

{1: 2, 2: 1}

### Multiplication

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

This code sets $n$ to $x \cdot y$.

In [14]:
arith_mul = [[1, -1, [2, -2, 3, 4], [4, -4, 2]], [2, -2], [3, -3, 1]]

In [15]:
fast_evaluate({1: 2, 2: 4}, arith_mul)

{1: 8}

### Division

*Input*: $2^x \cdot 3^y$. *Output*: $2^n \cdot 3^k$.

If $x \geq y$, given $a = qb + r$ where $a$ is input through $x$ and $b$ is input through $y$, it will calculate $q$ into $n$ and $r$ into $k$. Otherwise, it calculates $y - x$.

In [16]:
arith_div = [
  [2, -2, 7], # store r2 in r7
  [1,         # while r1>0
    [7, -7, 2, 8], # bring back r2
    [8, -8, 7]]    # save r2 to r7
    + arith_sub +  # see "Composing code"
    [9,            # increase quotient
    # if sub. underflow store rem in r8
    [2, [2, -2], [1, -1, 8], -9]],
  [7, -7],    # flush tmp
  [9, -9, 1], # set quotient
  [8, -8, 2]  # set remainder
]

The idea is to keep subtracting until subtraction returns 1 in $r_2$ (second register), which means underflow. We keep track of the number of subtractions and then set $r_2 = r_2 - r_1$ for the remainder.

In [17]:
fast_evaluate({1: 4, 2: 2}, arith_div)

{1: 2}

In [18]:
fast_evaluate({1: 4, 2: 3}, arith_div)

{2: 1, 1: 1}

### Exponents

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

This code sets $n$ to $x^y$. Requires $y>0$.

In [19]:
arith_exp = [
  -2,
  [2, -2, 5],      # move r2 to r5
  [1, -1, 6, 7],   # set r6, r7 to r1
  [7, -7, 1],      # move r7 to r1
  [5, -5,          # while y>0
    [6, -6, 7, 2],   # set r7, r2 to r6
    [7, -7, 6]]      # bring r6 back
    + arith_mul,     # multiply
  [6, -6]]         # flush r6

In [20]:
fast_evaluate({1: 2, 2: 3}, arith_exp)

{1: 8}

## Logic gates

In [21]:
logic_not = [2, [1, -1, -2], [2, -2, 1]]
logic_and = [[1, [1, -1], 3], [2, [2, -2], 3], -3, [3, -3, 1]]
logic_or = [[1, [1, -1], 3], [2, [2, -2], 3], [3, [3, -3], 1]]

In [22]:
for i in range(0, 3):
  print('not %d = ' % i, fast_evaluate({1: i}, logic_not))

not 0 =  {1: 1}
not 1 =  {}
not 2 =  {}


In [23]:
for i in range(0, 3):
  for j in range(0, 3):
    print('%d and %d =' % (i, j), fast_evaluate({1: i, 2: j}, logic_and))

0 and 0 = {}
0 and 1 = {}
0 and 2 = {}
1 and 0 = {}
1 and 1 = {1: 1}
1 and 2 = {1: 1}
2 and 0 = {}
2 and 1 = {1: 1}
2 and 2 = {1: 1}


In [24]:
for i in range(0, 3):
  for j in range(0, 3):
    print('%d or %d =' % (i, j), fast_evaluate({1: i, 2: j}, logic_or))

0 or 0 = {}
0 or 1 = {1: 1}
0 or 2 = {1: 1}
1 or 0 = {1: 1}
1 or 1 = {1: 1}
1 or 2 = {1: 1}
2 or 0 = {1: 1}
2 or 1 = {1: 1}
2 or 2 = {1: 1}


## Misc. algorithms

### Composing code

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

The Nand gate can be implemented by composing the `logic_and` and then the `logic_not` gate.

For example, in the following code, we simply merge the two lists:

In [25]:
logic_nand = logic_and + logic_not

In [26]:
for i in range(0, 2):
  for j in range(0, 2):
    print('%d nand %d =' % (i, j), fast_evaluate({1: i, 2: j}, logic_nand))

0 nand 0 = {1: 1}
0 nand 1 = {1: 1}
1 nand 0 = {1: 1}
1 nand 1 = {}


When composing code make sure you have made a copy of all the affected registers (if needed) since if there are overlapping registers some of the data may be overwritten.

### Fibonacci

*Input*: $2^x$. *Output*: $2^n$.

The following program sets $n$ to $Fib(x)$.

In [27]:
fib = [
  3,    # r1 = input/output, r2 = 0, r3 = 1, r4 = 0, r5 = 0
  [1,   # while n > 1
    -1,            # decrease n
    [3, -3, 2, 4], # move r3 to r4, and add it to r2
    [2, -2, 5],    # move r2 to r5
    [4, -4, 2],    # move r4 to r2, and add it to r2
    [5, -5, 3]],   # move r5 to r3
  [3, -3],    # flush r3
  [2, -2, 1]] # move r2 to r1

In [28]:
for i in range(0, 7):
  print('Fib(%d) =' % i, fast_evaluate({1: i}, fib))

Fib(0) = {}
Fib(1) = {1: 1}
Fib(2) = {1: 1}
Fib(3) = {1: 2}
Fib(4) = {1: 3}
Fib(5) = {1: 5}
Fib(6) = {1: 8}


### GCD

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

Sets $n$ to $GCD(x, y)$.

In [29]:
gcd = [
  [2,
    [2, -2, 11, 12], # copy b=r2 to r11, r12
    [12, -12, 2]]    # bring back b
    + arith_div +
    [[1, -1], [11, -11, 1]]] # set a = b
    # b is already remainder

In [30]:
fast_evaluate({1: 2, 2: 4}, gcd)

{1: 2}

In [31]:
fast_evaluate({1: 3, 2: 5}, gcd)

{1: 1}

In [32]:
fast_evaluate({1: 12, 2: 16}, gcd)

{1: 4}

### Primality test

*Input*: $2^x$. *Output*: $2^n$.

Sets $n$ to 1 if $x$ is prime, and 0 otherwise. Requires $x>1$.

In [33]:
is_prime = [
  [1, -1, 11, 12],    # copy r1 to r11/r12
  [11,
    [1, -1],
    [12, -12, 1, 15], # first argument is original input
    [15, -15, 12],    # save original value from r15 (tmp) to r12
    [2, -2],
    [11, -11, 15],    # copy r11 to r15
    [15, -15, 2, 11]] # second argument is current iterator
    + arith_div +     # do the division
    [
    # negate r2 (see logical gate "Not")
    [3, -3], 3, [2, -2, -3], [3, -3, 2],
    # if there was a non-zero remainder, it is now zero
    [2, -2, 14],      # if the remainder was zero, then it will be one and added to 14
    -11,
  ],
  [12, -12],             # flush r12 (original input)
  [1, -1], [14, -14, 1], # move result to r1
  # at this point, r1 contains the number of factors
  # if it's a prime, the number of factors will be two.
  -1, -1,                # if prime, r1 = 0, else r1 > 1
  # negate r1 for final result
  [2, -2], 2, [1, -1, -2], [2, -2, 1]]

The idea is to store in $r_{12}$ the result of `!(n%i)` for `i=n;i>=0`. We say that a number is prime if $r_{12}=2$, i.e. only two numbers divide the number (1 and itself).

In [34]:
for i in range(2, 10):
  print('is_prime(%d) =' % i, fast_evaluate({1: i}, is_prime))

is_prime(2) = {1: 1}
is_prime(3) = {1: 1}
is_prime(4) = {}
is_prime(5) = {1: 1}
is_prime(6) = {}
is_prime(7) = {1: 1}
is_prime(8) = {}
is_prime(9) = {}


### Logarithm

*Input*: $2^x \cdot 3^y$. *Output*: $2^n$.

Sets $n$ to $log_y(x)$.

In [35]:
logn = [
  [2, -2, 12], # store r2 to r12
  [1, # while the first argument is non-zero
    [2, -2],          # flush r2
    [15, -15],        # flush r15
    [12, -12, 2, 15], # restore r2 and save it to r3
    [15, -15, 12]]    # restore r12
    + arith_div +     # divide r1 by r2
    # negate r2 (see logical gate "Not")
    [[3, -3], 3, [2, -2, -3], [3, -3, 2],
    11,               # increase division count
  ],
  -11, # no off by one error
  [11, -11, 1], # flush r11 and move to r1
  [12, -12]]    # flush r12

In [36]:
for i in range(2, 9):
  print('log2(%d) =' % i, fast_evaluate({1: i, 2: 2}, logn))
  print('log3(%d) =' % (i+1), fast_evaluate({1: i+1, 2: 3}, logn))

log2(2) = {1: 1}
log3(3) = {1: 1}
log2(3) = {1: 1}
log3(4) = {1: 1}
log2(4) = {1: 2}
log3(5) = {1: 1}
log2(5) = {1: 2}
log3(6) = {1: 1}
log2(6) = {1: 2}
log3(7) = {1: 1}
log2(7) = {1: 2}
log3(8) = {1: 1}
log2(8) = {1: 3}
log3(9) = {1: 2}
